<a href="https://colab.research.google.com/github/jjefferson2006/DSWP_JJ_altera/blob/main/desafio_analise5_dumy_rg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycaret

     |████████████████████████████████| 256kB 8.5MB/s 
     |████████████████████████████████| 13.9MB 322kB/s 
     |████████████████████████████████| 1.8MB 40.8MB/s 
     |████████████████████████████████| 174kB 51.7MB/s 
     |████████████████████████████████| 102kB 8.5MB/s 
     |████████████████████████████████| 6.8MB 19.3MB/s 
     |████████████████████████████████| 1.6MB 37.5MB/s 
     |████████████████████████████████| 266kB 48.0MB/s 
     |████████████████████████████████| 148.9MB 80kB/s 
     |████████████████████████████████| 66.2MB 61kB/s 
     |████████████████████████████████| 276kB 47.5MB/s 
     |████████████████████████████████| 81kB 10.3MB/s 
     |████████████████████████████████| 163kB 53.7MB/s 
     |████████████████████████████████| 153kB 47.5MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 1.1MB 40.4MB/s 
     |████████████████████████████████| 337kB 43.2MB/s 
     |████████████████████████████████| 2.1MB 40.4MB/s

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, accuracy_score, confusion_matrix

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from pycaret.classification import *

In [118]:
#função pro cross_val
def funcao_cross_val_score(modelo, X_treinamento, y_treinamento, CV):
    # versão com sklearn.model_selection.cross_validate:
    #a_scores_CV = cross_validate(modelo, X_treinamento, y_treinamento, cv = CV, scoring = metodo)
    #print(f'Média das Acurácias calculadas pelo CV....: {100*round(a_scores_CV.mean(),4)}')
    #print(f'std médio das Acurácias calculadas pelo CV: {100*round(a_scores_CV.std(),4)}')
    #return a_scores_CV

    #versão com cross_val_score::
    a_scores_CV = cross_val_score(modelo, X_treinamento, y_treinamento, cv = CV)
    print(f'Média das Acurácias calculadas pelo CV....: {100*round(a_scores_CV.mean(),4)}')
    print(f'std médio das Acurácias calculadas pelo CV: {100*round(a_scores_CV.std(),4)}')
    return a_scores_CV

In [ ]:
#grid para cV
def GridSearchOptimizer(modelo, ml_Opt, d_hiperparametros, X_treinamento, y_treinamento, X_teste, y_teste, i_CV, l_colunas):
    ml_GridSearchCV = GridSearchCV(modelo, d_hiperparametros, cv = i_CV, n_jobs = -1, verbose= 10, scoring = 'accuracy')
    start = time()
    ml_GridSearchCV.fit(X_treinamento, y_treinamento)
    tempo_elapsed = time()-start
    print(f"\nGridSearchCV levou {tempo_elapsed:.2f} segundos.")

    # Hiperparâmetros que otimizam a classificação:
    print(f'\nHiperparâmetros otimizados: {ml_GridSearchCV.best_params_}')
    
    if ml_Opt == 'ml_DT2':
        print(f'\nDecisionTreeClassifier *********************************************************************************************************')
        ml_Opt = DecisionTreeClassifier(criterion= ml_GridSearchCV.best_params_['criterion'], 
                                        max_depth= ml_GridSearchCV.best_params_['max_depth'],
                                        max_leaf_nodes= ml_GridSearchCV.best_params_['max_leaf_nodes'],
                                        min_samples_split= ml_GridSearchCV.best_params_['min_samples_leaf'],
                                        min_samples_leaf= ml_GridSearchCV.best_params_['min_samples_split'], 
                                        random_state= i_Seed)
        
    elif ml_Opt == 'ml_RF2':
        print(f'\nRandomForestClassifier *********************************************************************************************************')
        ml_Opt = RandomForestClassifier(bootstrap= ml_GridSearchCV.best_params_['bootstrap'], 
                                        max_depth= ml_GridSearchCV.best_params_['max_depth'],
                                        max_features= ml_GridSearchCV.best_params_['max_features'],
                                        min_samples_leaf= ml_GridSearchCV.best_params_['min_samples_leaf'],
                                        min_samples_split= ml_GridSearchCV.best_params_['min_samples_split'],
                                        n_estimators= ml_GridSearchCV.best_params_['n_estimators'],
                                        random_state= i_Seed)
        
    elif ml_Opt == 'ml_AB2':
        print(f'\nAdaBoostClassifier *********************************************************************************************************')
        ml_Opt = AdaBoostClassifier(algorithm='SAMME.R', 
                                    base_estimator=RandomForestClassifier(bootstrap = False, 
                                                                          max_depth = 10, 
                                                                          max_features = 'auto', 
                                                                          min_samples_leaf = 1, 
                                                                          min_samples_split = 2, 
                                                                          n_estimators = 400), 
                                    learning_rate = ml_GridSearchCV.best_params_['learning_rate'], 
                                    n_estimators = ml_GridSearchCV.best_params_['n_estimators'], 
                                    random_state = i_Seed)
        
    elif ml_Opt == 'ml_GB2':
        print(f'\nGradientBoostingClassifier *********************************************************************************************************')
        ml_Opt = GradientBoostingClassifier(learning_rate = ml_GridSearchCV.best_params_['learning_rate'], 
                                            n_estimators = ml_GridSearchCV.best_params_['n_estimators'], 
                                            max_depth = ml_GridSearchCV.best_params_['max_depth'], 
                                            min_samples_split = ml_GridSearchCV.best_params_['min_samples_split'], 
                                            min_samples_leaf = ml_GridSearchCV.best_params_['min_samples_leaf'], 
                                            max_features = ml_GridSearchCV.best_params_['max_features'])
        
    elif ml_Opt == 'ml_XGB2':
        print(f'\nXGBoostingClassifier *********************************************************************************************************')
        ml_Opt = XGBoostingClassifier(learning_rate= ml_GridSearchCV.best_params_['learning_rate'], 
                                      max_depth= ml_GridSearchCV.best_params_['max_depth'], 
                                      colsample_bytree= ml_GridSearchCV.best_params_['colsample_bytree'], 
                                      subsample= ml_GridSearchCV.best_params_['subsample'], 
                                      gamma= ml_GridSearchCV.best_params_['gamma'], 
                                      min_child_weight= ml_GridSearchCV.best_params_['min_child_weight'])
        
    # Treina novamente usando os hiperparâmetros otimizados...
    ml_Opt.fit(X_treinamento, y_treinamento)

    # Cross-Validation com 10 folds
    print(f'\n********* CROSS-VALIDATION ***********')
    a_scores_CV = funcao_cross_val_score(ml_Opt, X_treinamento, y_treinamento, i_CV)

    # Faz predições com os hiperparâmetros otimizados...
    y_pred = ml_Opt.predict(X_teste)
  
    # Importância das COLUNAS
    print(f'\n********* IMPORTÂNCIA DAS COLUNAS ***********')
    df_importancia_variaveis = pd.DataFrame(zip(l_colunas, ml_Opt.feature_importances_), columns= ['coluna', 'importancia'])
    df_importancia_variaveis = df_importancia_variaveis.sort_values(by= ['importancia'], ascending=False)
    print(df_importancia_variaveis)

    # Matriz de Confusão
    print(f'\n********* CONFUSION MATRIX - PARAMETER TUNNING ***********')
    cf_matrix = confusion_matrix(y_teste, y_pred)
    cf_labels = ['True_Negative', 'False_Positive', 'False_Negative', 'True_Positive']
    cf_categories = ['Zero', 'One']
    mostra_confusion_matrix(cf_matrix, group_names = cf_labels, categories = cf_categories)

    return ml_Opt, ml_GridSearchCV.best_params_

In [ ]:
#coluna relevantes
from sklearn.feature_selection import SelectFromModel

def seleciona_colunas_relevantes(modelo, X_treinamento, X_teste, threshold = 0.05):
    # Cria um seletor para selecionar as COLUNAS com importância > threshold
    sfm = SelectFromModel(modelo, threshold)
    
    # Treina o seletor
    sfm.fit(X_treinamento, y_treinamento)

    # Mostra o indice das COLUNAS mais importantes
    print(f'\n********** COLUNAS Relevantes ******')
    print(sfm.get_support(indices=True))

    # Seleciona somente as COLUNAS relevantes
    X_treinamento_I = sfm.transform(X_treinamento)
    X_teste_I = sfm.transform(X_teste)
    return X_treinamento_I, X_teste_I   

In [ ]:
#mostra colunas relevantes
def mostra_feature_importances(clf, X_treinamento, y_treinamento=None, 
                             top_n=10, figsize=(8,8), print_table=False, title="Feature Importances"):
    '''
    plot feature importances of a tree-based sklearn estimator
    
    Note: X_treinamento and y_treinamento are pandas DataFrames
    
    Note: Scikit-plot is a lovely package but I sometimes have issues
              1. flexibility/extendibility
              2. complicated models/datasets
          But for many situations Scikit-plot is the way to go
          see https://scikit-plot.readthedocs.io/en/latest/Quickstart.html
    
    Parameters
    ----------
        clf         (sklearn estimator) if not fitted, this routine will fit it
        
        X_treinamento     (pandas DataFrame)
        
        y_treinamento     (pandas DataFrame)  optional
                                        required only if clf has not already been fitted 
        
        top_n       (int)               Plot the top_n most-important features
                                        Default: 10
                                        
        figsize     ((int,int))         The physical size of the plot
                                        Default: (8,8)
        
        print_table (boolean)           If True, print out the table of feature importances
                                        Default: False
        
    Returns
    -------
        the pandas dataframe with the features and their importance
        
    Author
    ------
        George Fisher
    '''
    
    __name__ = "mostra_feature_importances"
    
    import pandas as pd
    import numpy  as np
    import matplotlib.pyplot as plt
    
    from xgboost.core     import XGBoostError
    from lightgbm.sklearn import LightGBMError
    
    try: 
        if not hasattr(clf, 'feature_importances_'):
            clf.fit(X_treinamento.values, y_treinamento.values.ravel())

            if not hasattr(clf, 'feature_importances_'):
                raise AttributeError("{} does not have feature_importances_ attribute".
                                    format(clf.__class__.__name__))
                
    except (XGBoostError, LightGBMError, ValueError):
        clf.fit(X_treinamento.values, y_treinamento.values.ravel())
            
    feat_imp = pd.DataFrame({'importance':clf.feature_importances_})    
    feat_imp['feature'] = X_treinamento.columns
    feat_imp.sort_values(by ='importance', ascending = False, inplace = True)
    feat_imp = feat_imp.iloc[:top_n]
    
    feat_imp.sort_values(by='importance', inplace = True)
    feat_imp = feat_imp.set_index('feature', drop = True)
    feat_imp.plot.barh(title=title, figsize=figsize)
    plt.xlabel('Feature Importance Score')
    plt.show()
    
    if print_table:
        from IPython.display import display
        print("Top {} features in descending order of importance".format(top_n))
        display(feat_imp.sort_values(by = 'importance', ascending = False))
        
    return feat_imp

In [4]:
url = 'https://raw.githubusercontent.com/jjefferson2006/DSWP_JJ_altera/main/train.csv'
df_churn_train = pd.read_csv(url)
df_churn_train

,id,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,4030,Female,0,No,No,56.0,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,NaN,45.05,2560.1,0
1,6731,Male,0,Yes,Yes,NaN,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,332.65,0
2,6479,Female,0,Yes,No,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.7,0
3,6861,Female,0,No,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,1
4,3266,Male,0,Yes,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,6684,Male,0,No,No,43.0,No,No phone service,DSL,No,No,Yes,Yes,No,Yes,One year,No,Electronic check,44.15,1931.3,0
5630,604,Male,0,No,No,NaN,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,No,Electronic check,108.65,4903.2,0
5631,4803,Female,0,No,No,28.0,Yes,No,DSL,No,Yes,No,Yes,No,No,Month-to-month,No,Bank transfer (automatic),54.65,1517.5,0
5632,6317,Male,0,Yes,Yes,63.0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.25,1573.05,0


In [5]:
#Normalizar as colunas
df_churn_train.columns = [col.lower() for col in df_churn_train.columns]
df_churn_train.head()

,id,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,4030,Female,0,No,No,56.0,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,NaN,45.05,2560.1,0
1,6731,Male,0,Yes,Yes,NaN,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,332.65,0
2,6479,Female,0,Yes,No,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.7,0
3,6861,Female,0,No,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,1
4,3266,Male,0,Yes,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.8,1


In [6]:
df_churn_train.dtypes

id                    int64
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure              float64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
churn                 int64
dtype: object

In [7]:
df_churn_train.isna().sum()

id                    0
gender                0
seniorcitizen         0
partner               0
dependents          218
tenure              461
phoneservice          0
multiplelines         0
internetservice       0
onlinesecurity        0
onlinebackup          0
deviceprotection      0
techsupport           0
streamingtv           0
streamingmovies       0
contract              0
paperlessbilling      0
paymentmethod        99
monthlycharges        0
totalcharges          0
churn                 0
dtype: int64

In [8]:
df_churn_train.shape

(5634, 21)

###pay

In [9]:
#contar a variavel 
df_churn_train['paymentmethod'].value_counts()

Electronic check             1887
Mailed check                 1272
Bank transfer (automatic)    1212
Credit card (automatic)      1164
Name: paymentmethod, dtype: int64

In [10]:
df_churn_train['paymentmethod'].isna().sum()

99

In [11]:
df_churn_train['paymentmethod'].fillna('Credit card (automatic)', inplace=True)

###trat dep

In [12]:
df_churn_train['dependents'].isna().sum()

218

In [13]:
#contar a variavel 
df_churn_train['dependents'].value_counts()

No     3793
Yes    1623
Name: dependents, dtype: int64

In [14]:
#criterio = moda
df_churn_train['dependents'].fillna('Yes', inplace=True)

###tratar as numeric

In [15]:
df_churn_train['totalcharges'] = df_churn_train['totalcharges'].str.replace(',', '.')
df_churn_train['totalcharges'] = pd.to_numeric(df_churn_train['totalcharges'], errors= 'coerce')



In [16]:
df_churn_train['monthlycharges'].isna().sum()

0

In [17]:
df_churn_train['totalcharges'].sum()

12790788.7

In [18]:
df_churn_train['tenure'].isna().sum()

461

In [19]:
df_churn_train['tenure_esti']=round(df_churn_train['totalcharges']/df_churn_train['monthlycharges'],2)

In [20]:
df_churn_train['tenure_rou']=round(df_churn_train['tenure_esti'])

In [21]:
df_churn_train.head()

,id,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn,tenure_esti,tenure_rou
0,4030,Female,0,No,No,56.0,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,Credit card (automatic),45.05,2560.10,0,56.83,57.0
1,6731,Male,0,Yes,Yes,NaN,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,332.65,0,16.93,17.0
2,6479,Female,0,Yes,No,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.70,0,59.93,60.0
3,6861,Female,0,No,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,1,34.79,35.0
4,3266,Male,0,Yes,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.80,1,27.91,28.0


In [22]:
l_tot_ten = list(df_churn_train[df_churn_train['tenure'].isna()].index)
l_tot_ten

[1,
 11,
 34,
 45,
 55,
 92,
 94,
 109,
 121,
 133,
 185,
 190,
 193,
 200,
 221,
 223,
 231,
 234,
 247,
 251,
 268,
 288,
 298,
 323,
 332,
 346,
 353,
 368,
 372,
 381,
 390,
 398,
 408,
 409,
 436,
 453,
 463,
 464,
 465,
 469,
 472,
 474,
 483,
 485,
 486,
 508,
 520,
 544,
 559,
 560,
 572,
 596,
 602,
 620,
 632,
 640,
 682,
 700,
 707,
 773,
 787,
 791,
 809,
 811,
 830,
 832,
 851,
 854,
 880,
 886,
 923,
 925,
 950,
 956,
 958,
 968,
 988,
 992,
 1003,
 1049,
 1070,
 1090,
 1098,
 1105,
 1114,
 1115,
 1127,
 1135,
 1137,
 1143,
 1159,
 1169,
 1173,
 1192,
 1196,
 1206,
 1208,
 1214,
 1233,
 1238,
 1254,
 1261,
 1265,
 1266,
 1278,
 1279,
 1286,
 1336,
 1373,
 1405,
 1406,
 1414,
 1420,
 1440,
 1449,
 1475,
 1476,
 1490,
 1501,
 1503,
 1516,
 1526,
 1563,
 1584,
 1587,
 1591,
 1614,
 1637,
 1655,
 1681,
 1691,
 1695,
 1696,
 1703,
 1716,
 1718,
 1725,
 1734,
 1742,
 1750,
 1753,
 1770,
 1778,
 1789,
 1793,
 1802,
 1813,
 1817,
 1830,
 1845,
 1872,
 1887,
 1896,
 1906,
 1922,
 

In [23]:
for i in l_tot_ten:
    df_churn_train['tenure'][i] = df_churn_train['tenure_rou'][i]


In [24]:
df_churn_train.head()

,id,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn,tenure_esti,tenure_rou
0,4030,Female,0,No,No,56.0,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,Credit card (automatic),45.05,2560.10,0,56.83,57.0
1,6731,Male,0,Yes,Yes,17.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,332.65,0,16.93,17.0
2,6479,Female,0,Yes,No,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.70,0,59.93,60.0
3,6861,Female,0,No,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,1,34.79,35.0
4,3266,Male,0,Yes,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.80,1,27.91,28.0


In [25]:
df_churn_train.drop(columns=['tenure_esti','tenure_rou'], inplace=True)

In [26]:
df_churn_train.head()

,id,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,4030,Female,0,No,No,56.0,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,Credit card (automatic),45.05,2560.10,0
1,6731,Male,0,Yes,Yes,17.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,332.65,0
2,6479,Female,0,Yes,No,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.70,0
3,6861,Female,0,No,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,1
4,3266,Male,0,Yes,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.80,1


In [27]:
df_churn_train.columns

Index(['id', 'gender', 'seniorcitizen', 'partner', 'dependents', 'tenure',
       'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity',
       'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv',
       'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod',
       'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [28]:
df_churn_train['gender'].value_counts()

Male      2849
Female    2785
Name: gender, dtype: int64

In [29]:
df_churn_train['onlinesecurity'].value_counts()

No                     2763
Yes                    1647
No internet service    1224
Name: onlinesecurity, dtype: int64

In [30]:
df_churn_train['multiplelines'].value_counts()

No                  2692
Yes                 2390
No phone service     552
Name: multiplelines, dtype: int64

####daqui aplicar no teste

In [31]:
df_churn_train.onlinesecurity.replace(['No internet service'], ['No'], inplace=True)

onlinebackup	deviceprotection	techsupport	streamingtv	streamingmovies

In [32]:
df_churn_train.multiplelines.replace(['No phone service'], ['No'], inplace=True)

In [33]:
df_churn_train.onlinebackup.replace(['No internet service'], ['No'], inplace=True)

In [34]:
df_churn_train.deviceprotection.replace(['No internet service'], ['No'], inplace=True)

In [35]:
df_churn_train.techsupport.replace(['No internet service'], ['No'], inplace=True)

In [36]:
df_churn_train.streamingtv.replace(['No internet service'], ['No'], inplace=True)

In [37]:
df_churn_train.streamingmovies.replace(['No internet service'], ['No'], inplace=True)

In [38]:
df_churn_train.head()

,id,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,4030,Female,0,No,No,56.0,No,No,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,Credit card (automatic),45.05,2560.10,0
1,6731,Male,0,Yes,Yes,17.0,Yes,No,No,No,No,No,No,No,No,One year,No,Mailed check,19.65,332.65,0
2,6479,Female,0,Yes,No,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.70,0
3,6861,Female,0,No,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,1
4,3266,Male,0,Yes,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.80,1


In [39]:
dummy = pd.get_dummies(df_churn_train[['gender', 'partner', 'dependents', 
       'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity',
       'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv',
       'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod']])
dummy

,gender_Female,gender_Male,partner_No,partner_Yes,dependents_No,dependents_Yes,phoneservice_No,phoneservice_Yes,multiplelines_No,multiplelines_Yes,internetservice_DSL,internetservice_Fiber optic,internetservice_No,onlinesecurity_No,onlinesecurity_Yes,onlinebackup_No,onlinebackup_Yes,deviceprotection_No,deviceprotection_Yes,techsupport_No,techsupport_Yes,streamingtv_No,streamingtv_Yes,streamingmovies_No,streamingmovies_Yes,contract_Month-to-month,contract_One year,contract_Two year,paperlessbilling_No,paperlessbilling_Yes,paymentmethod_Bank transfer (automatic),paymentmethod_Credit card (automatic),paymentmethod_Electronic check,paymentmethod_Mailed check
0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,1,1,0,1,0,0,0,1,1,0,0,1,0,0
1,0,1,0,1,0,1,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1
2,1,0,0,1,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0
3,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,1,0,0,1,0,1,1,0,0,0,1,1,0,0,0
4,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,0,1,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,1,1,0,0,1,0,1,0,1,0,0,0,1,0
5630,0,1,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,0,0,1,0
5631,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,0,1,1,0,0,1,1,0,1,0,1,0,0,1,0,1,0,0,0
5632,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,1,0,0,0,0,1


In [40]:
df_train_dumy = pd.concat([df_churn_train, dummy], axis = 1)
df_train_dumy.head()

,id,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn,gender_Female,gender_Male,partner_No,partner_Yes,dependents_No,dependents_Yes,phoneservice_No,phoneservice_Yes,multiplelines_No,multiplelines_Yes,internetservice_DSL,internetservice_Fiber optic,internetservice_No,onlinesecurity_No,onlinesecurity_Yes,onlinebackup_No,onlinebackup_Yes,deviceprotection_No,deviceprotection_Yes,techsupport_No,techsupport_Yes,streamingtv_No,streamingtv_Yes,streamingmovies_No,streamingmovies_Yes,contract_Month-to-month,contract_One year,contract_Two year,paperlessbilling_No,paperlessbilling_Yes,paymentmethod_Bank transfer (automatic),paymentmethod_Credit card (automatic),paymentmethod_Electronic check,paymentmethod_Mailed check
0,4030,Female,0,No,No,56.0,No,No,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,Credit card (automatic),45.05,2560.10,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,1,1,0,1,0,0,0,1,1,0,0,1,0,0
1,6731,Male,0,Yes,Yes,17.0,Yes,No,No,No,No,No,No,No,No,One year,No,Mailed check,19.65,332.65,0,0,1,0,1,0,1,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1
2,6479,Female,0,Yes,No,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.70,0,1,0,0,1,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0
3,6861,Female,0,No,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,1,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,1,0,0,1,0,1,1,0,0,0,1,1,0,0,0
4,3266,Male,0,Yes,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.80,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,1,0


In [ ]:
 'gender', 'partner', 'dependents', 'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity',
       'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv',
       'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod'

In [41]:
df_train_dumy.drop(columns=['gender', 'partner', 'dependents',
                            'phoneservice', 'multiplelines', 'internetservice',
                            'onlinesecurity', 'onlinebackup', 'deviceprotection'
                            ,'techsupport', 'streamingtv', 'streamingmovies',
                            'contract', 'paperlessbilling', 'paymentmethod'], inplace = True)

In [42]:
df_train_dumy

,id,seniorcitizen,tenure,monthlycharges,totalcharges,churn,gender_Female,gender_Male,partner_No,partner_Yes,dependents_No,dependents_Yes,phoneservice_No,phoneservice_Yes,multiplelines_No,multiplelines_Yes,internetservice_DSL,internetservice_Fiber optic,internetservice_No,onlinesecurity_No,onlinesecurity_Yes,onlinebackup_No,onlinebackup_Yes,deviceprotection_No,deviceprotection_Yes,techsupport_No,techsupport_Yes,streamingtv_No,streamingtv_Yes,streamingmovies_No,streamingmovies_Yes,contract_Month-to-month,contract_One year,contract_Two year,paperlessbilling_No,paperlessbilling_Yes,paymentmethod_Bank transfer (automatic),paymentmethod_Credit card (automatic),paymentmethod_Electronic check,paymentmethod_Mailed check
0,4030,0,56.0,45.05,2560.10,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,1,1,0,1,0,0,0,1,1,0,0,1,0,0
1,6731,0,17.0,19.65,332.65,0,0,1,0,1,0,1,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1
2,6479,0,60.0,110.80,6640.70,0,1,0,0,1,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0
3,6861,0,37.0,101.90,3545.35,1,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,1,0,0,1,0,1,1,0,0,0,1,1,0,0,0
4,3266,0,29.0,70.75,1974.80,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,6684,0,43.0,44.15,1931.30,0,0,1,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,1,1,0,0,1,0,1,0,1,0,0,0,1,0
5630,604,0,45.0,108.65,4903.20,0,0,1,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,0,0,1,0
5631,4803,0,28.0,54.65,1517.50,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,0,1,1,0,0,1,1,0,1,0,1,0,0,1,0,1,0,0,0
5632,6317,0,63.0,25.25,1573.05,0,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,1,0,0,0,0,1


In [43]:
l_index_charg = list(df_train_dumy[df_train_dumy['totalcharges'].isna()].index)
l_index_charg

[161, 416, 1766, 2145, 3375, 3404, 4008, 5151]

In [44]:
for i in l_index_charg:
    df_train_dumy['totalcharges'][i] = 0.0

In [45]:
df_train_dumy.isna().sum()

id                                         0
seniorcitizen                              0
tenure                                     0
monthlycharges                             0
totalcharges                               0
churn                                      0
gender_Female                              0
gender_Male                                0
partner_No                                 0
partner_Yes                                0
dependents_No                              0
dependents_Yes                             0
phoneservice_No                            0
phoneservice_Yes                           0
multiplelines_No                           0
multiplelines_Yes                          0
internetservice_DSL                        0
internetservice_Fiber optic                0
internetservice_No                         0
onlinesecurity_No                          0
onlinesecurity_Yes                         0
onlinebackup_No                            0
onlineback

In [46]:
df_train_dumy

,id,seniorcitizen,tenure,monthlycharges,totalcharges,churn,gender_Female,gender_Male,partner_No,partner_Yes,dependents_No,dependents_Yes,phoneservice_No,phoneservice_Yes,multiplelines_No,multiplelines_Yes,internetservice_DSL,internetservice_Fiber optic,internetservice_No,onlinesecurity_No,onlinesecurity_Yes,onlinebackup_No,onlinebackup_Yes,deviceprotection_No,deviceprotection_Yes,techsupport_No,techsupport_Yes,streamingtv_No,streamingtv_Yes,streamingmovies_No,streamingmovies_Yes,contract_Month-to-month,contract_One year,contract_Two year,paperlessbilling_No,paperlessbilling_Yes,paymentmethod_Bank transfer (automatic),paymentmethod_Credit card (automatic),paymentmethod_Electronic check,paymentmethod_Mailed check
0,4030,0,56.0,45.05,2560.10,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,1,1,0,1,0,0,0,1,1,0,0,1,0,0
1,6731,0,17.0,19.65,332.65,0,0,1,0,1,0,1,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1
2,6479,0,60.0,110.80,6640.70,0,1,0,0,1,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0
3,6861,0,37.0,101.90,3545.35,1,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,1,0,0,1,0,1,1,0,0,0,1,1,0,0,0
4,3266,0,29.0,70.75,1974.80,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,6684,0,43.0,44.15,1931.30,0,0,1,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,1,1,0,0,1,0,1,0,1,0,0,0,1,0
5630,604,0,45.0,108.65,4903.20,0,0,1,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,0,0,1,0
5631,4803,0,28.0,54.65,1517.50,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,0,1,1,0,0,1,1,0,1,0,1,0,0,1,0,1,0,0,0
5632,6317,0,63.0,25.25,1573.05,0,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,1,0,0,0,0,1


In [96]:
df_train = df_train_dumy.copy()
df_train.head()

,id,seniorcitizen,tenure,monthlycharges,totalcharges,churn,gender_Female,gender_Male,partner_No,partner_Yes,dependents_No,dependents_Yes,phoneservice_No,phoneservice_Yes,multiplelines_No,multiplelines_Yes,internetservice_DSL,internetservice_Fiber optic,internetservice_No,onlinesecurity_No,onlinesecurity_Yes,onlinebackup_No,onlinebackup_Yes,deviceprotection_No,deviceprotection_Yes,techsupport_No,techsupport_Yes,streamingtv_No,streamingtv_Yes,streamingmovies_No,streamingmovies_Yes,contract_Month-to-month,contract_One year,contract_Two year,paperlessbilling_No,paperlessbilling_Yes,paymentmethod_Bank transfer (automatic),paymentmethod_Credit card (automatic),paymentmethod_Electronic check,paymentmethod_Mailed check
0,4030,0,56.0,45.05,2560.10,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,1,1,0,1,0,0,0,1,1,0,0,1,0,0
1,6731,0,17.0,19.65,332.65,0,0,1,0,1,0,1,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1
2,6479,0,60.0,110.80,6640.70,0,1,0,0,1,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0
3,6861,0,37.0,101.90,3545.35,1,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,1,0,0,1,0,1,1,0,0,0,1,1,0,0,0
4,3266,0,29.0,70.75,1974.80,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,1,0


In [137]:
df_train.shape

(5634, 39)

In [97]:
df_y_rg = df_train['churn']

In [98]:
df_train.drop(columns = 'churn', inplace = True)

In [99]:
df_X_rg = df_train

In [140]:
df_X_rg.shape

(5634, 39)

In [143]:
df_X_rg.head()

,id,seniorcitizen,tenure,monthlycharges,totalcharges,gender_Female,gender_Male,partner_No,partner_Yes,dependents_No,dependents_Yes,phoneservice_No,phoneservice_Yes,multiplelines_No,multiplelines_Yes,internetservice_DSL,internetservice_Fiber optic,internetservice_No,onlinesecurity_No,onlinesecurity_Yes,onlinebackup_No,onlinebackup_Yes,deviceprotection_No,deviceprotection_Yes,techsupport_No,techsupport_Yes,streamingtv_No,streamingtv_Yes,streamingmovies_No,streamingmovies_Yes,contract_Month-to-month,contract_One year,contract_Two year,paperlessbilling_No,paperlessbilling_Yes,paymentmethod_Bank transfer (automatic),paymentmethod_Credit card (automatic),paymentmethod_Electronic check,paymentmethod_Mailed check
0,4030,0,56.0,45.05,2560.10,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,1,1,0,1,0,0,0,1,1,0,0,1,0,0
1,6731,0,17.0,19.65,332.65,0,1,0,1,0,1,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1
2,6479,0,60.0,110.80,6640.70,1,0,0,1,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0
3,6861,0,37.0,101.90,3545.35,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,1,0,0,1,0,1,1,0,0,0,1,1,0,0,0
4,3266,0,29.0,70.75,1974.80,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,1,0


In [144]:
df_X_rg.columns

Index(['id', 'seniorcitizen', 'tenure', 'monthlycharges', 'totalcharges',
       'gender_Female', 'gender_Male', 'partner_No', 'partner_Yes',
       'dependents_No', 'dependents_Yes', 'phoneservice_No',
       'phoneservice_Yes', 'multiplelines_No', 'multiplelines_Yes',
       'internetservice_DSL', 'internetservice_Fiber optic',
       'internetservice_No', 'onlinesecurity_No', 'onlinesecurity_Yes',
       'onlinebackup_No', 'onlinebackup_Yes', 'deviceprotection_No',
       'deviceprotection_Yes', 'techsupport_No', 'techsupport_Yes',
       'streamingtv_No', 'streamingtv_Yes', 'streamingmovies_No',
       'streamingmovies_Yes', 'contract_Month-to-month', 'contract_One year',
       'contract_Two year', 'paperlessbilling_No', 'paperlessbilling_Yes',
       'paymentmethod_Bank transfer (automatic)',
       'paymentmethod_Credit card (automatic)',
       'paymentmethod_Electronic check', 'paymentmethod_Mailed check'],
      dtype='object')

In [146]:
df_X_rg.drop(columns = ['id', 'totalcharges',
       'onlinesecurity_No', 'onlinesecurity_Yes',
       'onlinebackup_No', 'deviceprotection_No',
       'techsupport_No','paperlessbilling_No', 'paperlessbilling_Yes',
       'paymentmethod_Electronic check', 'paymentmethod_Mailed check'], inplace = True)

In [148]:
df_X_rg.head()

,seniorcitizen,tenure,monthlycharges,gender_Female,gender_Male,partner_No,partner_Yes,dependents_No,dependents_Yes,phoneservice_No,phoneservice_Yes,multiplelines_No,multiplelines_Yes,internetservice_DSL,internetservice_Fiber optic,internetservice_No,onlinebackup_Yes,deviceprotection_Yes,techsupport_Yes,streamingtv_No,streamingtv_Yes,streamingmovies_No,streamingmovies_Yes,contract_Month-to-month,contract_One year,contract_Two year,paymentmethod_Bank transfer (automatic),paymentmethod_Credit card (automatic)
0,0,56.0,45.05,1,0,1,0,1,0,1,0,1,0,1,0,0,1,1,1,1,0,1,0,0,0,1,0,1
1,0,17.0,19.65,0,1,0,1,0,1,0,1,1,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0
2,0,60.0,110.80,1,0,0,1,1,0,0,1,0,1,0,1,0,1,1,1,0,1,0,1,0,0,1,0,1
3,0,37.0,101.90,1,0,1,0,1,0,0,1,0,1,0,1,0,1,1,0,0,1,0,1,1,0,0,1,0
4,0,29.0,70.75,0,1,0,1,0,1,0,1,1,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0


In [149]:
df_X_rg.shape

(5634, 28)

In [150]:
from sklearn.model_selection import train_test_split
X_treinamento_rg, X_teste_rg, y_treinamento_rg, y_teste_rg = train_test_split(df_X_rg, df_y_rg, test_size = 0.2)

In [151]:
modelo2 = LogisticRegression(penalty='none', solver='newton-cg')


In [152]:
modelo2.fit(X_treinamento_rg, y_treinamento_rg)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='none',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [153]:
cv_i=10

In [122]:
from sklearn.metrics import accuracy_score # para medir a acurácia do modelo preditivo
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix # para plotar a confusion matrix

from sklearn.model_selection import GridSearchCV # para otimizar os hiperparâmetros dos modelos preditivos
from sklearn.model_selection import cross_val_score # Para o CV (Cross-Validation)
from sklearn.model_selection import cross_validate

from time import time
from operator import itemgetter
from scipy.stats import randint

from sklearn.tree import export_graphviz
#from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus

np.set_printoptions(suppress=True)

In [154]:
a_scores_CV = funcao_cross_val_score(modelo2, X_treinamento_rg, y_treinamento_rg, CV=cv_i)

Média das Acurácias calculadas pelo CV....: 80.41
std médio das Acurácias calculadas pelo CV: 2.16


In [155]:
a_scores_CV

array([0.83592018, 0.78713969, 0.79379157, 0.78935698, 0.8248337 ,
       0.76940133, 0.83370288, 0.78888889, 0.79777778, 0.82      ])

In [156]:
print(f'Acurácias: {a_scores_CV}')

Acurácias: [0.83592018 0.78713969 0.79379157 0.78935698 0.8248337  0.76940133
 0.83370288 0.78888889 0.79777778 0.82      ]


In [157]:
modelo2.score(X_teste_rg,y_teste_rg)

0.7923691215616682

In [158]:
X_treinamento = X_treinamento_rg
X_teste = X_teste_rg
y_treinamento = y_treinamento_rg

In [159]:
#coluna relevantes
from sklearn.feature_selection import SelectFromModel

def seleciona_colunas_relevantes(modelo, X_treinamento, X_teste, threshold = 0.05):
    # Cria um seletor para selecionar as COLUNAS com importância > threshold
    sfm = SelectFromModel(modelo, threshold)
    
    # Treina o seletor
    sfm.fit(X_treinamento, y_treinamento)

    # Mostra o indice das COLUNAS mais importantes
    print(f'\n********** COLUNAS Relevantes ******')
    print(sfm.get_support(indices=True))

    # Seleciona somente as COLUNAS relevantes
    X_treinamento_I = sfm.transform(X_treinamento)
    X_teste_I = sfm.transform(X_teste)
    return X_treinamento_I, X_teste_I   

In [160]:
seleciona_colunas_relevantes(modelo2, X_treinamento,X_teste)


********** COLUNAS Relevantes ******
[ 0  2  3  4  5  6  7  8  9 10 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27]


(array([[ 0.  , 20.55,  0.  , ...,  0.  ,  0.  ,  1.  ],
        [ 0.  , 80.8 ,  0.  , ...,  1.  ,  1.  ,  0.  ],
        [ 1.  , 75.45,  1.  , ...,  0.  ,  0.  ,  0.  ],
        ...,
        [ 0.  , 30.5 ,  0.  , ...,  0.  ,  0.  ,  0.  ],
        [ 0.  , 33.7 ,  0.  , ...,  0.  ,  0.  ,  1.  ],
        [ 0.  , 19.8 ,  0.  , ...,  0.  ,  0.  ,  0.  ]]),
 array([[ 0.  , 23.95,  0.  , ...,  0.  ,  0.  ,  1.  ],
        [ 0.  , 70.7 ,  1.  , ...,  0.  ,  0.  ,  0.  ],
        [ 0.  , 45.85,  0.  , ...,  0.  ,  0.  ,  0.  ],
        ...,
        [ 0.  , 19.8 ,  1.  , ...,  1.  ,  0.  ,  0.  ],
        [ 1.  , 90.85,  1.  , ...,  0.  ,  0.  ,  1.  ],
        [ 0.  , 56.15,  0.  , ...,  0.  ,  1.  ,  0.  ]]))

In [104]:
y_pred = modelo2.predict_proba(X_teste_rg)

In [105]:
y_pred

array([[0.16261957, 0.83738043],
       [0.6036915 , 0.3963085 ],
       [0.78210107, 0.21789893],
       ...,
       [0.89937752, 0.10062248],
       [0.55030944, 0.44969056],
       [0.94494199, 0.05505801]])

In [112]:
pre = modelo2.predict(X_teste_rg)
pre

array([1, 0, 0, ..., 0, 0, 0])

In [113]:
sco = modelo2.score(X_teste_rg, y_teste_rg)
sco

0.7976929902395741

In [48]:
df_train

,id,seniorcitizen,tenure,monthlycharges,totalcharges,churn,gender_Female,gender_Male,partner_No,partner_Yes,dependents_No,dependents_Yes,phoneservice_No,phoneservice_Yes,multiplelines_No,multiplelines_Yes,internetservice_DSL,internetservice_Fiber optic,internetservice_No,onlinesecurity_No,onlinesecurity_Yes,onlinebackup_No,onlinebackup_Yes,deviceprotection_No,deviceprotection_Yes,techsupport_No,techsupport_Yes,streamingtv_No,streamingtv_Yes,streamingmovies_No,streamingmovies_Yes,contract_Month-to-month,contract_One year,contract_Two year,paperlessbilling_No,paperlessbilling_Yes,paymentmethod_Bank transfer (automatic),paymentmethod_Credit card (automatic),paymentmethod_Electronic check,paymentmethod_Mailed check
0,4030,0,56.0,45.05,2560.10,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,1,1,0,1,0,0,0,1,1,0,0,1,0,0
1,6731,0,17.0,19.65,332.65,0,0,1,0,1,0,1,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1
2,6479,0,60.0,110.80,6640.70,0,1,0,0,1,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0
3,6861,0,37.0,101.90,3545.35,1,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,1,0,0,1,0,1,1,0,0,0,1,1,0,0,0
4,3266,0,29.0,70.75,1974.80,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,6684,0,43.0,44.15,1931.30,0,0,1,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,1,1,0,0,1,0,1,0,1,0,0,0,1,0
5630,604,0,45.0,108.65,4903.20,0,0,1,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,0,0,1,0
5631,4803,0,28.0,54.65,1517.50,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,0,1,1,0,0,1,1,0,1,0,1,0,0,1,0,1,0,0,0
5632,6317,0,63.0,25.25,1573.05,0,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,1,0,0,0,0,1


In [ ]:
df_y = df_train['churn']

In [ ]:
df_y

0       0
1       0
2       0
3       1
4       1
       ..
5629    0
5630    0
5631    0
5632    0
5633    0
Name: churn, Length: 5634, dtype: int64

In [ ]:
df_train.drop(columns = 'churn', inplace = True)

In [ ]:
df_train.head()

,id,seniorcitizen,tenure,monthlycharges,totalcharges,gender_Female,gender_Male,partner_No,partner_Yes,dependents_No,dependents_Yes,phoneservice_No,phoneservice_Yes,multiplelines_No,multiplelines_Yes,internetservice_DSL,internetservice_Fiber optic,internetservice_No,onlinesecurity_No,onlinesecurity_Yes,onlinebackup_No,onlinebackup_Yes,deviceprotection_No,deviceprotection_Yes,techsupport_No,techsupport_Yes,streamingtv_No,streamingtv_Yes,streamingmovies_No,streamingmovies_Yes,contract_Month-to-month,contract_One year,contract_Two year,paperlessbilling_No,paperlessbilling_Yes,paymentmethod_Bank transfer (automatic),paymentmethod_Credit card (automatic),paymentmethod_Electronic check,paymentmethod_Mailed check
0,4030,0,56.0,45.05,2560.10,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,1,1,0,1,0,0,0,1,1,0,0,1,0,0
1,6731,0,17.0,19.65,332.65,0,1,0,1,0,1,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1
2,6479,0,60.0,110.80,6640.70,1,0,0,1,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0
3,6861,0,37.0,101.90,3545.35,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,1,0,0,1,0,1,1,0,0,0,1,1,0,0,0
4,3266,0,29.0,70.75,1974.80,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,1,0


In [ ]:
df_X = df_train

In [ ]:
df_X

,id,seniorcitizen,tenure,monthlycharges,totalcharges,gender_Female,gender_Male,partner_No,partner_Yes,dependents_No,dependents_Yes,phoneservice_No,phoneservice_Yes,multiplelines_No,multiplelines_Yes,internetservice_DSL,internetservice_Fiber optic,internetservice_No,onlinesecurity_No,onlinesecurity_Yes,onlinebackup_No,onlinebackup_Yes,deviceprotection_No,deviceprotection_Yes,techsupport_No,techsupport_Yes,streamingtv_No,streamingtv_Yes,streamingmovies_No,streamingmovies_Yes,contract_Month-to-month,contract_One year,contract_Two year,paperlessbilling_No,paperlessbilling_Yes,paymentmethod_Bank transfer (automatic),paymentmethod_Credit card (automatic),paymentmethod_Electronic check,paymentmethod_Mailed check
0,4030,0,56.0,45.05,2560.10,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,1,1,0,1,0,0,0,1,1,0,0,1,0,0
1,6731,0,17.0,19.65,332.65,0,1,0,1,0,1,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1
2,6479,0,60.0,110.80,6640.70,1,0,0,1,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0
3,6861,0,37.0,101.90,3545.35,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,1,0,0,1,0,1,1,0,0,0,1,1,0,0,0
4,3266,0,29.0,70.75,1974.80,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,6684,0,43.0,44.15,1931.30,0,1,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,1,1,0,0,1,0,1,0,1,0,0,0,1,0
5630,604,0,45.0,108.65,4903.20,0,1,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,0,0,1,0
5631,4803,0,28.0,54.65,1517.50,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,0,1,1,0,0,1,1,0,1,0,1,0,0,1,0,1,0,0,0
5632,6317,0,63.0,25.25,1573.05,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,1,0,0,0,0,1


In [ ]:
from sklearn.model_selection import train_test_split
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(df_X, df_y, test_size = 0.3)

In [ ]:
X_treinamento.shape

(3943, 39)

In [ ]:
y_treinamento.shape

(3943,)

In [ ]:
X_teste.shape

(1691, 39)

In [ ]:
y_teste.shape

(1691,)

In [ ]:
#primeira rodada
from sklearn.ensemble import GradientBoostingClassifier

# Instancia...
ml_GB = GradientBoostingClassifier(n_estimators = 100, min_samples_split = 2)

# Treina... 
ml_GB.fit(X_treinamento, y_treinamento)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
cv_i = 10

In [ ]:
from sklearn.metrics import accuracy_score # para medir a acurácia do modelo preditivo
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix # para plotar a confusion matrix

from sklearn.model_selection import GridSearchCV # para otimizar os hiperparâmetros dos modelos preditivos
from sklearn.model_selection import cross_val_score # Para o CV (Cross-Validation)
from sklearn.model_selection import cross_validate

from time import time
from operator import itemgetter
from scipy.stats import randint

from sklearn.tree import export_graphviz
#from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus

np.set_printoptions(suppress=True)

In [ ]:
# Cross-Validation com 10 folds
a_scores_CV = funcao_cross_val_score(ml_GB, X_treinamento, y_treinamento, CV=cv_i)

Média das Acurácias calculadas pelo CV....: 80.78
std médio das Acurácias calculadas pelo CV: 0.89


In [ ]:
a_scores_CV 

array([0.81265823, 0.81012658, 0.81772152, 0.80964467, 0.8071066 ,
       0.80964467, 0.7893401 , 0.79441624, 0.8071066 , 0.81979695])

In [ ]:
a_scores_CV.mean()

0.8077562166677374

In [ ]:
print(f'Acurácias: {a_scores_CV}')

Acurácias: [0.81265823 0.81012658 0.81772152 0.80964467 0.8071066  0.80964467
 0.7893401  0.79441624 0.8071066  0.81979695]


In [ ]:
ml_GB.score(X_teste, y_teste)

0.8066232998225902

In [ ]:
df_importantes = pd.DataFrame(zip(X_treinamento.columns, ml_GB.feature_importances_), columns = ['coluna', 'importancia'])
df_importantes.sort_values(by = 'importancia', ascending = False)

,coluna,importancia
30,contract_Month-to-month,0.370740
2,tenure,0.138623
16,internetservice_Fiber optic,0.115909
4,totalcharges,0.102167
3,monthlycharges,0.083185
37,paymentmethod_Electronic check,0.054820
0,id,0.029531
33,paperlessbilling_No,0.010933
34,paperlessbilling_Yes,0.009500
17,internetservice_No,0.009328


In [ ]:
df_importantes.columns

Index(['coluna', 'importancia'], dtype='object')

In [ ]:
df_importantes[df_importantes['importancia']>0.008]

,coluna,importancia
0,id,0.029531
2,tenure,0.138623
3,monthlycharges,0.083185
4,totalcharges,0.102167
16,internetservice_Fiber optic,0.115909
17,internetservice_No,0.009328
30,contract_Month-to-month,0.370740
33,paperlessbilling_No,0.010933
34,paperlessbilling_Yes,0.009500
37,paymentmethod_Electronic check,0.054820


In [ ]:
df_importantes['importancia']<0.008

0     False
1      True
2     False
3     False
4     False
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16    False
17    False
18     True
19     True
20     True
21     True
22     True
23     True
24     True
25     True
26     True
27     True
28     True
29     True
30    False
31     True
32     True
33    False
34    False
35     True
36     True
37    False
38     True
Name: importancia, dtype: bool

In [ ]:
a = len(df_importantes['coluna'])
l_colunas_s =[]
for i in range(a):
    l_colunas_s.append(df_importantes['coluna'][i])         


In [ ]:
l_colunas_s

['id',
 'seniorcitizen',
 'tenure',
 'monthlycharges',
 'totalcharges',
 'gender_Female',
 'gender_Male',
 'partner_No',
 'partner_Yes',
 'dependents_No',
 'dependents_Yes',
 'phoneservice_No',
 'phoneservice_Yes',
 'multiplelines_No',
 'multiplelines_Yes',
 'internetservice_DSL',
 'internetservice_Fiber optic',
 'internetservice_No',
 'onlinesecurity_No',
 'onlinesecurity_Yes',
 'onlinebackup_No',
 'onlinebackup_Yes',
 'deviceprotection_No',
 'deviceprotection_Yes',
 'techsupport_No',
 'techsupport_Yes',
 'streamingtv_No',
 'streamingtv_Yes',
 'streamingmovies_No',
 'streamingmovies_Yes',
 'contract_Month-to-month',
 'contract_One year',
 'contract_Two year',
 'paperlessbilling_No',
 'paperlessbilling_Yes',
 'paymentmethod_Bank transfer (automatic)',
 'paymentmethod_Credit card (automatic)',
 'paymentmethod_Electronic check',
 'paymentmethod_Mailed check']

In [ ]:
#primeiro drop de colunas
df_X.drop(columns = [ 'seniorcitizen', 'gender_Female', 'gender_Male', 'partner_No', 'partner_Yes',
                    'dependents_No', 'dependents_Yes', 'phoneservice_No', 'phoneservice_Yes',
                    'multiplelines_No',  'multiplelines_Yes', 'internetservice_DSL', 'onlinesecurity_No',
                    'onlinesecurity_Yes', 'onlinebackup_No', 'onlinebackup_Yes', 'deviceprotection_No',
                    'deviceprotection_Yes', 'techsupport_No', 'techsupport_Yes', 'streamingtv_No',
                    'streamingtv_Yes', 'streamingmovies_No', 'streamingmovies_Yes', 'contract_One year',
                    'contract_Two year', 'paymentmethod_Bank transfer (automatic)', 'paymentmethod_Credit card (automatic)',
                    'paymentmethod_Mailed check'], inplace = True)

In [ ]:
df_X.columns

Index(['id', 'tenure', 'monthlycharges', 'totalcharges',
       'internetservice_Fiber optic', 'internetservice_No',
       'contract_Month-to-month', 'paperlessbilling_No',
       'paperlessbilling_Yes', 'paymentmethod_Electronic check'],
      dtype='object')

In [ ]:
#divide a amostra novamente

In [ ]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(df_X, df_y, test_size = 0.3)

In [ ]:
df_X.head()

,id,tenure,monthlycharges,totalcharges,internetservice_Fiber optic,internetservice_No,contract_Month-to-month,paperlessbilling_No,paperlessbilling_Yes,paymentmethod_Electronic check
0,4030,56.0,45.05,2560.10,0,0,0,1,0,0
1,6731,17.0,19.65,332.65,0,1,0,1,0,0
2,6479,60.0,110.80,6640.70,1,0,0,0,1,0
3,6861,37.0,101.90,3545.35,1,0,1,0,1,0
4,3266,29.0,70.75,1974.80,1,0,1,0,1,1


In [ ]:
#segunda passada
ml_GB.fit(X_treinamento, y_treinamento)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
a_scores_CV = funcao_cross_val_score(ml_GB, X_treinamento, y_treinamento, CV=cv_i)

Média das Acurácias calculadas pelo CV....: 79.3
std médio das Acurácias calculadas pelo CV: 2.08


In [ ]:
a_scores_CV 

array([0.8       , 0.81265823, 0.78734177, 0.78426396, 0.78426396,
       0.80964467, 0.78172589, 0.80964467, 0.81725888, 0.74365482])

In [ ]:
ml_GB.score(X_teste, y_teste)

0.8172678888231816

In [ ]:
df_importantes = pd.DataFrame(zip(X_treinamento.columns, ml_GB.feature_importances_), columns = ['coluna', 'importancia'])
df_importantes.sort_values(by = 'importancia', ascending = False)

,coluna,importancia
6,contract_Month-to-month,0.430170
1,tenure,0.170345
3,totalcharges,0.095503
4,internetservice_Fiber optic,0.093899
2,monthlycharges,0.091650
0,id,0.046963
9,paymentmethod_Electronic check,0.032413
8,paperlessbilling_Yes,0.015840
5,internetservice_No,0.014508
7,paperlessbilling_No,0.008708


In [ ]:
l_colunas = df_X.columns
l_colunas

Index(['id', 'tenure', 'monthlycharges', 'totalcharges',
       'internetservice_Fiber optic', 'internetservice_No',
       'contract_Month-to-month', 'paperlessbilling_No',
       'paperlessbilling_Yes', 'paymentmethod_Electronic check'],
      dtype='object')

In [ ]:
i_CV=10

In [ ]:
# Dicionário de hiperparâmetros para o parameter tunning.
#d_hiperparametros_GB= {'learning_rate': [1, 0.5, 0.25, 0.1, 0.05, 0.01]} #,
#                  'n_estimators': [1, 2, 4, 8, 16, 32, 64, 100, 200],
#                  'max_depth': [5, 10, 15, 20, 25, 30],
#                  'min_samples_split': [0.1, 0.3, 0.5, 0.7, 0.9],
#                  'min_samples_leaf': [0.1, 0.2, 0.3, 0.4, 0.5],
#                  'max_features': list(range(1, X_treinamento.shape[1]))}

In [ ]:
d_hiperparametros_GB= {'learning_rate': [1, 0.5],
                  'n_estimators': [10, 100, 200],
                  'max_depth': [ 20, 25, 30],
                  'min_samples_split': [0.1, 0.3, 0.5],
                  'min_samples_leaf': [0.1, 0.2],
                  'max_features': list(range(1, X_treinamento.shape[1]))}

In [ ]:
# Invoca a função
ml_GB2, best_params= GridSearchOptimizer(ml_GB, 'ml_GB2', d_hiperparametros_GB, X_treinamento, y_treinamento, X_teste, y_teste, i_CV, l_colunas)

Fitting 10 folds for each of 972 candidates, totalling 9720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.2000s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done 116 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done 146 tas


GridSearchCV levou 2273.11 segundos.

Hiperparâmetros otimizados: {'learning_rate': 0.5, 'max_depth': 20, 'max_features': 5, 'min_samples_leaf': 0.2, 'min_samples_split': 0.5, 'n_estimators': 10}

GradientBoostingClassifier *********************************************************************************************************

********* CROSS-VALIDATION ***********
Média das Acurácias calculadas pelo CV....: 79.41
std médio das Acurácias calculadas pelo CV: 1.92

********* IMPORTÂNCIA DAS COLUNAS ***********
                           coluna  importancia
1                          tenure     0.362611
6         contract_Month-to-month     0.282210
4     internetservice_Fiber optic     0.133227
2                  monthlycharges     0.098828
9  paymentmethod_Electronic check     0.071119
5              internetservice_No     0.017692
3                    totalcharges     0.015225
8            paperlessbilling_Yes     0.011711
7             paperlessbilling_No     0.007377
0            

NameError: ignored

In [ ]:
best_params: {'learning_rate': 0.5, 'max_depth': 20, 'max_features': 5,
               'min_samples_leaf': 0.2, 'min_samples_split': 0.5, 'n_estimators': 10}


In [ ]:
'''ml_Opt

In [ ]:
'''ml_GridSearchCV.best_params_

In [ ]:
df_X.columns

Index(['id', 'tenure', 'monthlycharges', 'totalcharges',
       'internetservice_Fiber optic', 'internetservice_No',
       'contract_Month-to-month', 'paperlessbilling_No',
       'paperlessbilling_Yes', 'paymentmethod_Electronic check'],
      dtype='object')

In [ ]:
df_X.drop(columns = ['id'], inplace = True)

In [ ]:
#paperlessbilling_No,paperlessbilling_Yes

In [ ]:
df_X.drop(columns = ['paperlessbilling_No','paperlessbilling_Yes'], inplace = True)

In [ ]:
#colunas definitivas
df_X.columns

Index(['tenure', 'monthlycharges', 'totalcharges',
       'internetservice_Fiber optic', 'internetservice_No',
       'contract_Month-to-month', 'paymentmethod_Electronic check'],
      dtype='object')

In [ ]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(df_X, df_y, test_size = 0.3)

In [ ]:
#best_params: {'learning_rate': 0.5, 'max_depth': 20, 'max_features': 5,
#              'min_samples_leaf': 0.2, 'min_samples_split': 0.5, 'n_estimators': 10}

ml_GB2= GradientBoostingClassifier(learning_rate= 0.5, max_depth= 20, max_features=5
                                   , min_samples_leaf= 0.2, min_samples_split= 0.5
                                   , n_estimators= 10)

In [ ]:
#modelo best
ml_GB2.fit(X_treinamento, y_treinamento)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.5, loss='deviance', max_depth=20,
                           max_features=5, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=0.2, min_samples_split=0.5,
                           min_weight_fraction_leaf=0.0, n_estimators=10,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
a_scores_CV = funcao_cross_val_score(ml_GB2, X_treinamento, y_treinamento, CV=cv_i)

Média das Acurácias calculadas pelo CV....: 79.58
std médio das Acurácias calculadas pelo CV: 1.63


In [ ]:
a_scores_CV 

array([0.78227848, 0.81265823, 0.7721519 , 0.8071066 , 0.78680203,
       0.79187817, 0.8071066 , 0.77918782, 0.79187817, 0.82741117])

In [ ]:
#aplica no teste
ml_GB2.score(X_teste, y_teste)

0.8083973979893554

In [ ]:
y_pred_GB2 = ml_GB2.predict(X_teste)


In [ ]:
#calcula a accuracy
accuracy_score(y_teste, y_pred_GB2)

0.8083973979893554

In [ ]:
'dependents_No', 'partner_Yes', 'deviceprotection_Yes','gender_Female',
                    'deviceprotection_No', 'paymentmethod_Bank transfer (automatic)','phoneservice_Yes',
                    'dependents_Yes', 'onlinebackup_No','paymentmethod_Mailed check', 'onlinesecurity_Yes',
                    'partner_No', 'streamingmovies_No', 'paymentmethod_Credit card (automatic)',
                    'multiplelines_Yes', 'gender_Male', 'onlinebackup_Yes', 'phoneservice_No', 'techsupport_Yes'

In [ ]:
'''#segundo drop
df_X.drop(columns = ['dependents_No', 'gender_Female', 'paymentmethod_Bank transfer (automatic)','phoneservice_Yes',
                    'paymentmethod_Mailed check', 'onlinesecurity_Yes', 'partner_No', 'streamingmovies_No',
                    'paymentmethod_Credit card (automatic)', 'multiplelines_Yes', 'gender_Male',
                    'onlinebackup_Yes', 'phoneservice_No', 'techsupport_Yes'], inplace = True)

In [ ]:
'''X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(df_X, df_y, test_size = 0.3)

In [ ]:
'''#terceira passada
ml_GB.fit(X_treinamento, y_treinamento)

GradientBoostingClassifier()

In [ ]:
'''a_scores_CV = funcao_cross_val_score(ml_GB, X_treinamento, y_treinamento, CV=cv_i)

Média das Acurácias calculadas pelo CV....: 81.34
std médio das Acurácias calculadas pelo CV: 1.91


In [ ]:
'''a_scores_CV 

array([0.81012658, 0.80759494, 0.76202532, 0.82741117, 0.80964467,
       0.82233503, 0.8248731 , 0.81218274, 0.82233503, 0.83502538])

In [ ]:
'''ml_GB.score(X_teste, y_teste)

0.7989355410999409

In [ ]:
'''df_importantes = pd.DataFrame(zip(X_treinamento.columns, ml_GB.feature_importances_), columns = ['coluna', 'importancia'])
df_importantes.sort_values(by = 'importancia', ascending = False)

,coluna,importancia
30,contract_Month-to-month,0.379513
2,tenure,0.144347
16,internetservice_Fiber optic,0.118077
4,totalcharges,0.097449
3,monthlycharges,0.068735
37,paymentmethod_Electronic check,0.046521
0,id,0.037891
17,internetservice_No,0.021328
33,paperlessbilling_No,0.010943
1,seniorcitizen,0.010815


In [ ]:
'''df_X.head()

,id,seniorcitizen,tenure,monthlycharges,totalcharges,multiplelines_No,internetservice_DSL,internetservice_Fiber optic,internetservice_No,onlinesecurity_No,techsupport_No,streamingtv_Yes,streamingmovies_Yes,contract_Month-to-month,contract_One year,contract_Two year,paperlessbilling_No,paperlessbilling_Yes,paymentmethod_Electronic check
0,4030,0,56.0,45.05,2560.10,1,1,0,0,0,0,0,0,0,0,1,1,0,0
1,6731,0,17.0,19.65,332.65,1,0,0,1,1,1,0,0,0,1,0,1,0,0
2,6479,0,60.0,110.80,6640.70,0,0,1,0,1,0,1,1,0,0,1,0,1,0
3,6861,0,37.0,101.90,3545.35,0,0,1,0,1,1,1,1,1,0,0,0,1,0
4,3266,0,29.0,70.75,1974.80,1,0,1,0,1,1,0,0,1,0,0,0,1,1


In [ ]:
'''df_X.columns

Index(['id', 'seniorcitizen', 'tenure', 'monthlycharges', 'totalcharges',
       'multiplelines_No', 'internetservice_DSL',
       'internetservice_Fiber optic', 'internetservice_No',
       'onlinesecurity_No', 'techsupport_No', 'streamingtv_Yes',
       'streamingmovies_Yes', 'contract_Month-to-month', 'contract_One year',
       'contract_Two year', 'paperlessbilling_No', 'paperlessbilling_Yes',
       'paymentmethod_Electronic check'],
      dtype='object')

###trata categor

In [ ]:
'''df_churn_train.gender.replace(['Female', 'Male'], [1, 0], inplace=True)
df_churn_train.partner.replace(['Yes', 'No'], [1, 0], inplace=True)
df_churn_train.phoneservice.replace(['Yes', 'No'], [1, 0], inplace=True)
df_churn_train.paperlessbilling.replace(['Yes', 'No'], [1, 0], inplace=True)

In [ ]:
'''df_churn_train.internetservice.replace(['Fiber optic', 'DSL', 'No'], [1,2,0], inplace=True)
df_churn_train.paymentmethod.replace(['Electronic check', 'Bank transfer (automatic)', 'Mailed check', 'Credit card (automatic)'], [1,2,3,4], inplace=True)
df_churn_train.contract.replace(['Two year', 'One year', 'Month-to-month'], [0,1,2], inplace=True)
df_churn_train.multiplelines.replace(['No', 'Yes', 'No phone service'], [0,1,3], inplace=True)
df_churn_train.onlinesecurity.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True) 
df_churn_train.onlinebackup.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True)
df_churn_train.deviceprotection.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True)
df_churn_train.techsupport.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True)
df_churn_train.streamingtv.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True)
df_churn_train.streamingmovies.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True) 

In [ ]:
'''df_churn_train.dependents.replace(['Yes', 'No'], [1, 0], inplace=True)

In [ ]:
'''df_churn_train.head()

,id,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,4030,1,0,0,0,56.0,0,3,2,1,1,1,1,0,0,0,0,4,45.05,2560.10,0
1,6731,0,0,1,1,17.0,1,0,0,2,2,2,2,2,2,1,0,3,19.65,332.65,0
2,6479,1,0,1,0,60.0,1,1,1,0,1,1,1,1,1,0,1,4,110.80,6640.70,0
3,6861,1,0,0,0,37.0,1,1,1,0,1,1,0,1,1,2,1,2,101.90,3545.35,1
4,3266,0,0,1,1,29.0,1,0,1,0,0,0,0,0,0,2,1,1,70.75,1974.80,1


In [ ]:
'''df_churn_train.isna().sum()

id                  0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        8
churn               0
dtype: int64

#####tratar test

In [49]:
url = 'https://raw.githubusercontent.com/jjefferson2006/DSWP_JJ_altera/main/test.csv'
df_churn_test = pd.read_csv(url)
df_churn_test

,id,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,5027,Male,0,Yes,Yes,23.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Credit card (automatic),20.00,445.3
1,1733,Male,1,Yes,Yes,61.0,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,One year,No,NaN,99.00,5969.3
2,5384,Male,0,No,No,36.0,Yes,Yes,Fiber optic,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,84.75,3050.15
3,6554,Female,0,Yes,Yes,61.0,No,No phone service,DSL,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),61.45,3751.15
4,364,Female,0,No,No,47.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Mailed check,20.55,945.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,4897,Male,0,No,NaN,24.0,Yes,No,DSL,No,No,Yes,No,No,No,Month-to-month,Yes,Mailed check,49.70,1167.8
1405,6940,Male,0,No,No,35.0,Yes,No,Fiber optic,Yes,No,No,Yes,No,Yes,One year,Yes,Electronic check,89.20,3251.3
1406,804,Female,0,Yes,No,46.0,Yes,No,DSL,No,Yes,No,Yes,Yes,No,Two year,Yes,Credit card (automatic),64.20,3009.5
1407,1143,Male,1,Yes,Yes,11.0,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Credit card (automatic),75.20,775.3


In [50]:
#Normalizar as colunas
df_churn_test.columns = [col.lower() for col in df_churn_test.columns]
df_churn_test.head()

,id,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges
0,5027,Male,0,Yes,Yes,23.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Credit card (automatic),20.00,445.3
1,1733,Male,1,Yes,Yes,61.0,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,One year,No,NaN,99.00,5969.3
2,5384,Male,0,No,No,36.0,Yes,Yes,Fiber optic,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,84.75,3050.15
3,6554,Female,0,Yes,Yes,61.0,No,No phone service,DSL,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),61.45,3751.15
4,364,Female,0,No,No,47.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Mailed check,20.55,945.7


In [51]:
df_churn_test['dependents'].fillna('Yes', inplace=True)

In [52]:
df_churn_test['paymentmethod'].fillna('Credit card (automatic)', inplace=True)

In [53]:
df_churn_test['totalcharges'] = df_churn_test['totalcharges'].str.replace(',', '.')
df_churn_test['totalcharges'] = pd.to_numeric(df_churn_test['totalcharges'], errors= 'coerce')

In [54]:
df_churn_test[['totalcharges']].isna().sum()

totalcharges    3
dtype: int64

In [55]:
df_churn_test['tenure'].isna().sum()

118

In [ ]:
#atenção

In [56]:
df_churn_test['tenure_esti']=round(df_churn_test['totalcharges']/df_churn_test['monthlycharges'],2)

In [57]:
df_churn_test['tenure_rou']=round(df_churn_test['tenure_esti'])

In [58]:
l_tot_ten = list(df_churn_test[df_churn_test['tenure'].isna()].index)
l_tot_ten

[8,
 15,
 32,
 38,
 41,
 54,
 69,
 82,
 113,
 117,
 118,
 125,
 149,
 172,
 173,
 185,
 193,
 213,
 238,
 246,
 268,
 277,
 287,
 295,
 318,
 324,
 339,
 351,
 355,
 363,
 377,
 417,
 467,
 476,
 483,
 487,
 495,
 498,
 511,
 513,
 516,
 517,
 518,
 561,
 562,
 563,
 573,
 599,
 624,
 629,
 635,
 638,
 641,
 654,
 660,
 662,
 669,
 677,
 678,
 728,
 729,
 734,
 740,
 747,
 764,
 765,
 791,
 796,
 805,
 817,
 820,
 827,
 835,
 842,
 861,
 888,
 897,
 910,
 912,
 918,
 925,
 926,
 928,
 952,
 953,
 958,
 973,
 979,
 987,
 1009,
 1012,
 1022,
 1034,
 1045,
 1051,
 1069,
 1107,
 1159,
 1163,
 1169,
 1173,
 1189,
 1200,
 1213,
 1222,
 1224,
 1254,
 1276,
 1303,
 1310,
 1325,
 1344,
 1367,
 1368,
 1385,
 1387,
 1389,
 1394]

In [59]:
for i in l_tot_ten:
    df_churn_test['tenure'][i] = df_churn_test['tenure_rou'][i]


In [60]:
df_churn_test.drop(columns=['tenure_esti','tenure_rou'], inplace=True)

In [61]:
l_index_charg = list(df_churn_test[df_churn_test['totalcharges'].isna()].index)
l_index_charg

[313, 1063, 1297]

In [62]:
df_churn_test[['tenure','totalcharges']][313:314]

,tenure,totalcharges
313,0.0,NaN


In [63]:
for i in l_index_charg:
    df_churn_test['totalcharges'][i] = 0.0

In [64]:
df_churn_test.isna().sum()

id                  0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
dtype: int64

In [65]:
df_churn_test['totalcharges'][313]

0.0

In [71]:
df_churn_test['onlinesecurity'].value_counts()

No                     735
Yes                    372
No internet service    302
Name: onlinesecurity, dtype: int64

In [66]:
df_churn_test

,id,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges
0,5027,Male,0,Yes,Yes,23.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Credit card (automatic),20.00,445.30
1,1733,Male,1,Yes,Yes,61.0,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,One year,No,Credit card (automatic),99.00,5969.30
2,5384,Male,0,No,No,36.0,Yes,Yes,Fiber optic,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,84.75,3050.15
3,6554,Female,0,Yes,Yes,61.0,No,No phone service,DSL,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),61.45,3751.15
4,364,Female,0,No,No,47.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Mailed check,20.55,945.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,4897,Male,0,No,Yes,24.0,Yes,No,DSL,No,No,Yes,No,No,No,Month-to-month,Yes,Mailed check,49.70,1167.80
1405,6940,Male,0,No,No,35.0,Yes,No,Fiber optic,Yes,No,No,Yes,No,Yes,One year,Yes,Electronic check,89.20,3251.30
1406,804,Female,0,Yes,No,46.0,Yes,No,DSL,No,Yes,No,Yes,Yes,No,Two year,Yes,Credit card (automatic),64.20,3009.50
1407,1143,Male,1,Yes,Yes,11.0,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Credit card (automatic),75.20,775.30


In [72]:
df_churn_test.onlinesecurity.replace(['No internet service'],['No'],inplace=True)
df_churn_test.multiplelines.replace(['No phone service'],['No'],inplace=True)
df_churn_test.onlinebackup.replace(['No internet service'],['No'],inplace=True)
df_churn_test.deviceprotection.replace(['No internet service'],['No'],inplace=True)
df_churn_test.techsupport.replace(['No internet service'],['No'],inplace=True)
df_churn_test.streamingtv.replace(['No internet service'],['No'],inplace=True)
df_churn_test.streamingmovies.replace(['No internet service'],['No'],inplace=True)

In [74]:
df_churn_test.multiplelines.replace(['No phone service'],['No'],inplace=True)

In [75]:
df_churn_test.head()

,id,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges
0,5027,Male,0,Yes,Yes,23.0,Yes,No,No,No,No,No,No,No,No,One year,Yes,Credit card (automatic),20.00,445.30
1,1733,Male,1,Yes,Yes,61.0,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,One year,No,Credit card (automatic),99.00,5969.30
2,5384,Male,0,No,No,36.0,Yes,Yes,Fiber optic,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,84.75,3050.15
3,6554,Female,0,Yes,Yes,61.0,No,No,DSL,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),61.45,3751.15
4,364,Female,0,No,No,47.0,Yes,No,No,No,No,No,No,No,No,Two year,Yes,Mailed check,20.55,945.70


In [76]:
dummy_t=pd.get_dummies(df_churn_test[['gender','partner','dependents','phoneservice',
                                      'multiplelines','internetservice','onlinesecurity',
                                      'onlinebackup','deviceprotection','techsupport','streamingtv',
                                      'streamingmovies','contract','paperlessbilling','paymentmethod']])
dummy_t

,gender_Female,gender_Male,partner_No,partner_Yes,dependents_No,dependents_Yes,phoneservice_No,phoneservice_Yes,multiplelines_No,multiplelines_Yes,internetservice_DSL,internetservice_Fiber optic,internetservice_No,onlinesecurity_No,onlinesecurity_Yes,onlinebackup_No,onlinebackup_Yes,deviceprotection_No,deviceprotection_Yes,techsupport_No,techsupport_Yes,streamingtv_No,streamingtv_Yes,streamingmovies_No,streamingmovies_Yes,contract_Month-to-month,contract_One year,contract_Two year,paperlessbilling_No,paperlessbilling_Yes,paymentmethod_Bank transfer (automatic),paymentmethod_Credit card (automatic),paymentmethod_Electronic check,paymentmethod_Mailed check
0,0,1,0,1,0,1,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,0,1,0,1,0,0
1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,0
2,0,1,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,0,0,1,0,0,1,0
3,1,0,0,1,0,1,1,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,1,0,0,0
4,1,0,1,0,1,0,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,0,1,1,0,0,1,0,1,1,0,1,0,0,1,0,1,0,0,1,1,0,1,0,1,0,1,0,0,0,1,0,0,0,1
1405,0,1,1,0,1,0,0,1,1,0,0,1,0,0,1,1,0,1,0,0,1,1,0,0,1,0,1,0,0,1,0,0,1,0
1406,1,0,0,1,1,0,0,1,1,0,1,0,0,1,0,0,1,1,0,0,1,0,1,1,0,0,0,1,0,1,0,1,0,0
1407,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,0


In [77]:
df_test_dumy=pd.concat([df_churn_test,dummy_t],axis=1)
df_test_dumy

,id,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,gender_Female,gender_Male,partner_No,partner_Yes,dependents_No,dependents_Yes,phoneservice_No,phoneservice_Yes,multiplelines_No,multiplelines_Yes,internetservice_DSL,internetservice_Fiber optic,internetservice_No,onlinesecurity_No,onlinesecurity_Yes,onlinebackup_No,onlinebackup_Yes,deviceprotection_No,deviceprotection_Yes,techsupport_No,techsupport_Yes,streamingtv_No,streamingtv_Yes,streamingmovies_No,streamingmovies_Yes,contract_Month-to-month,contract_One year,contract_Two year,paperlessbilling_No,paperlessbilling_Yes,paymentmethod_Bank transfer (automatic),paymentmethod_Credit card (automatic),paymentmethod_Electronic check,paymentmethod_Mailed check
0,5027,Male,0,Yes,Yes,23.0,Yes,No,No,No,No,No,No,No,No,One year,Yes,Credit card (automatic),20.00,445.30,0,1,0,1,0,1,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,0,1,0,1,0,0
1,1733,Male,1,Yes,Yes,61.0,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,One year,No,Credit card (automatic),99.00,5969.30,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,0
2,5384,Male,0,No,No,36.0,Yes,Yes,Fiber optic,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,84.75,3050.15,0,1,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,0,0,1,0,0,1,0
3,6554,Female,0,Yes,Yes,61.0,No,No,DSL,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),61.45,3751.15,1,0,0,1,0,1,1,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,1,0,0,0
4,364,Female,0,No,No,47.0,Yes,No,No,No,No,No,No,No,No,Two year,Yes,Mailed check,20.55,945.70,1,0,1,0,1,0,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,4897,Male,0,No,Yes,24.0,Yes,No,DSL,No,No,Yes,No,No,No,Month-to-month,Yes,Mailed check,49.70,1167.80,0,1,1,0,0,1,0,1,1,0,1,0,0,1,0,1,0,0,1,1,0,1,0,1,0,1,0,0,0,1,0,0,0,1
1405,6940,Male,0,No,No,35.0,Yes,No,Fiber optic,Yes,No,No,Yes,No,Yes,One year,Yes,Electronic check,89.20,3251.30,0,1,1,0,1,0,0,1,1,0,0,1,0,0,1,1,0,1,0,0,1,1,0,0,1,0,1,0,0,1,0,0,1,0
1406,804,Female,0,Yes,No,46.0,Yes,No,DSL,No,Yes,No,Yes,Yes,No,Two year,Yes,Credit card (automatic),64.20,3009.50,1,0,0,1,1,0,0,1,1,0,1,0,0,1,0,0,1,1,0,0,1,0,1,1,0,0,0,1,0,1,0,1,0,0
1407,1143,Male,1,Yes,Yes,11.0,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Credit card (automatic),75.20,775.30,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,0


In [78]:
df_test_dumy.drop(columns=['gender','partner','dependents','phoneservice','multiplelines','internetservice',
                           'onlinesecurity','onlinebackup','deviceprotection','techsupport','streamingtv',
                           'streamingmovies','contract','paperlessbilling','paymentmethod'],inplace=True)

In [79]:
l_index_charg_t=list(df_test_dumy[df_test_dumy['totalcharges'].isna()].index)
l_index_charg_t

[]

In [80]:
df_test_dumy.isna().sum()

id                                         0
seniorcitizen                              0
tenure                                     0
monthlycharges                             0
totalcharges                               0
gender_Female                              0
gender_Male                                0
partner_No                                 0
partner_Yes                                0
dependents_No                              0
dependents_Yes                             0
phoneservice_No                            0
phoneservice_Yes                           0
multiplelines_No                           0
multiplelines_Yes                          0
internetservice_DSL                        0
internetservice_Fiber optic                0
internetservice_No                         0
onlinesecurity_No                          0
onlinesecurity_Yes                         0
onlinebackup_No                            0
onlinebackup_Yes                           0
deviceprot

In [81]:
df_test_dumy.head()

,id,seniorcitizen,tenure,monthlycharges,totalcharges,gender_Female,gender_Male,partner_No,partner_Yes,dependents_No,dependents_Yes,phoneservice_No,phoneservice_Yes,multiplelines_No,multiplelines_Yes,internetservice_DSL,internetservice_Fiber optic,internetservice_No,onlinesecurity_No,onlinesecurity_Yes,onlinebackup_No,onlinebackup_Yes,deviceprotection_No,deviceprotection_Yes,techsupport_No,techsupport_Yes,streamingtv_No,streamingtv_Yes,streamingmovies_No,streamingmovies_Yes,contract_Month-to-month,contract_One year,contract_Two year,paperlessbilling_No,paperlessbilling_Yes,paymentmethod_Bank transfer (automatic),paymentmethod_Credit card (automatic),paymentmethod_Electronic check,paymentmethod_Mailed check
0,5027,0,23.0,20.00,445.30,0,1,0,1,0,1,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,0,1,0,1,0,0
1,1733,1,61.0,99.00,5969.30,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,0
2,5384,0,36.0,84.75,3050.15,0,1,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,0,0,1,0,0,1,0
3,6554,0,61.0,61.45,3751.15,1,0,0,1,0,1,1,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,1,0,0,0
4,364,0,47.0,20.55,945.70,1,0,1,0,1,0,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,1


In [ ]:
#colunas que ficaram
df_X.columns

Index(['tenure', 'monthlycharges', 'totalcharges',
       'internetservice_Fiber optic', 'internetservice_No',
       'contract_Month-to-month', 'paymentmethod_Electronic check'],
      dtype='object')

In [ ]:
'id', 'seniorcitizen','gender_Female', 'gender_Male', 'partner_No', 'partner_Yes',
       'dependents_No', 'dependents_Yes', 'phoneservice_No',
       'phoneservice_Yes', 'multiplelines_No',
       'multiplelines_No phone service', 'multiplelines_Yes',
       'internetservice_DSL','onlinesecurity_No',
       'onlinesecurity_No internet service', 'onlinesecurity_Yes',
       'onlinebackup_No', 'onlinebackup_No internet service',
       'onlinebackup_Yes', 'deviceprotection_No',
       'deviceprotection_No internet service', 'deviceprotection_Yes',
       'techsupport_No', 'techsupport_No internet service', 'techsupport_Yes',
       'streamingtv_No', 'streamingtv_No internet service', 'streamingtv_Yes',
       'streamingmovies_No', 'streamingmovies_No internet service',
       'streamingmovies_Yes', 'contract_One year',
       'contract_Two year', 'paperlessbilling_No', 'paperlessbilling_Yes',
       'paymentmethod_Bank transfer (automatic)',
       'paymentmethod_Credit card (automatic)', 'paymentmethod_Mailed check'],
      dtype='object')


In [ ]:
df_test_dumy.columns

Index(['id', 'seniorcitizen', 'tenure', 'monthlycharges', 'totalcharges',
       'gender_Female', 'gender_Male', 'partner_No', 'partner_Yes',
       'dependents_No', 'dependents_Yes', 'phoneservice_No',
       'phoneservice_Yes', 'multiplelines_No',
       'multiplelines_No phone service', 'multiplelines_Yes',
       'internetservice_DSL', 'internetservice_Fiber optic',
       'internetservice_No', 'onlinesecurity_No',
       'onlinesecurity_No internet service', 'onlinesecurity_Yes',
       'onlinebackup_No', 'onlinebackup_No internet service',
       'onlinebackup_Yes', 'deviceprotection_No',
       'deviceprotection_No internet service', 'deviceprotection_Yes',
       'techsupport_No', 'techsupport_No internet service', 'techsupport_Yes',
       'streamingtv_No', 'streamingtv_No internet service', 'streamingtv_Yes',
       'streamingmovies_No', 'streamingmovies_No internet service',
       'streamingmovies_Yes', 'contract_Month-to-month', 'contract_One year',
       'contract_Two ye

In [ ]:
#primeiro drop de colunas
df_test_dumy.drop(columns = ['id','seniorcitizen','gender_Female', 'gender_Male', 'partner_No', 'partner_Yes',
                     'dependents_No', 'dependents_Yes', 'phoneservice_No',
                     'phoneservice_Yes', 'multiplelines_No',
                     'multiplelines_No phone service', 'multiplelines_Yes',
                     'internetservice_DSL', 'onlinesecurity_No',
                     'onlinesecurity_No internet service', 'onlinesecurity_Yes',
                     'onlinebackup_No', 'onlinebackup_No internet service',
                     'onlinebackup_Yes', 'deviceprotection_No',
                     'deviceprotection_No internet service', 'deviceprotection_Yes',
                     'techsupport_No', 'techsupport_No internet service', 'techsupport_Yes',
                     'streamingtv_No', 'streamingtv_No internet service', 'streamingtv_Yes',
                     'streamingmovies_No', 'streamingmovies_No internet service',
                     'streamingmovies_Yes', 'contract_One year',
                     'contract_Two year', 'paperlessbilling_No', 'paperlessbilling_Yes',
                     'paymentmethod_Bank transfer (automatic)','paymentmethod_Credit card (automatic)',
                     'paymentmethod_Mailed check'], inplace = True)

In [ ]:
df_test_dumy.columns

Index(['tenure', 'monthlycharges', 'totalcharges',
       'internetservice_Fiber optic', 'internetservice_No',
       'contract_Month-to-month', 'paymentmethod_Electronic check'],
      dtype='object')

In [ ]:
df_X.columns

Index(['tenure', 'monthlycharges', 'totalcharges',
       'internetservice_Fiber optic', 'internetservice_No',
       'contract_Month-to-month', 'paymentmethod_Electronic check'],
      dtype='object')

In [ ]:
y_pred_GB2 = ml_GB2.predict(df_test_dumy)

In [ ]:
y_pred_GB2

array([0, 0, 0, ..., 0, 1, 0])

In [ ]:
pred_val = y_pred_GB2
pred_val

array([0, 0, 0, ..., 0, 1, 0])

In [ ]:
df_test_dumy.index

RangeIndex(start=0, stop=1409, step=1)

In [ ]:
from google.colab import files

In [ ]:
#ids=df_test_dumy['id']
ids = df_churn_test['id']
output=pd.DataFrame({'id':ids,'Churn':pred_val})
output.to_csv('submission2GB2.csv',index=False)
files.download('submission2GB2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
'''df_churn_test.gender.replace(['Female', 'Male'], [1, 0], inplace=True)
df_churn_test.partner.replace(['Yes', 'No'], [1, 0], inplace=True)
df_churn_test.phoneservice.replace(['Yes', 'No'], [1, 0], inplace=True)
df_churn_test.paperlessbilling.replace(['Yes', 'No'], [1, 0], inplace=True)

In [ ]:
'''df_churn_test.internetservice.replace(['Fiber optic', 'DSL', 'No'], [1,2,0], inplace=True)
df_churn_test.paymentmethod.replace(['Electronic check', 'Bank transfer (automatic)', 'Mailed check', 'Credit card (automatic)'], [1,2,3,4], inplace=True)
df_churn_test.contract.replace(['Two year', 'One year', 'Month-to-month'], [0,1,2], inplace=True)
df_churn_test.multiplelines.replace(['No', 'Yes', 'No phone service'], [0,1,3], inplace=True)
df_churn_test.onlinesecurity.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True) 
df_churn_test.onlinebackup.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True)
df_churn_test.deviceprotection.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True)
df_churn_test.techsupport.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True)
df_churn_test.streamingtv.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True)
df_churn_test.streamingmovies.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True) 


###drop no treino

In [83]:
#treino
df_train_dumy.columns

Index(['id', 'seniorcitizen', 'tenure', 'monthlycharges', 'totalcharges',
       'churn', 'gender_Female', 'gender_Male', 'partner_No', 'partner_Yes',
       'dependents_No', 'dependents_Yes', 'phoneservice_No',
       'phoneservice_Yes', 'multiplelines_No', 'multiplelines_Yes',
       'internetservice_DSL', 'internetservice_Fiber optic',
       'internetservice_No', 'onlinesecurity_No', 'onlinesecurity_Yes',
       'onlinebackup_No', 'onlinebackup_Yes', 'deviceprotection_No',
       'deviceprotection_Yes', 'techsupport_No', 'techsupport_Yes',
       'streamingtv_No', 'streamingtv_Yes', 'streamingmovies_No',
       'streamingmovies_Yes', 'contract_Month-to-month', 'contract_One year',
       'contract_Two year', 'paperlessbilling_No', 'paperlessbilling_Yes',
       'paymentmethod_Bank transfer (automatic)',
       'paymentmethod_Credit card (automatic)',
       'paymentmethod_Electronic check', 'paymentmethod_Mailed check'],
      dtype='object')

In [85]:
#test
df_test_dumy.columns

Index(['id', 'seniorcitizen', 'tenure', 'monthlycharges', 'totalcharges',
       'gender_Female', 'gender_Male', 'partner_No', 'partner_Yes',
       'dependents_No', 'dependents_Yes', 'phoneservice_No',
       'phoneservice_Yes', 'multiplelines_No', 'multiplelines_Yes',
       'internetservice_DSL', 'internetservice_Fiber optic',
       'internetservice_No', 'onlinesecurity_No', 'onlinesecurity_Yes',
       'onlinebackup_No', 'onlinebackup_Yes', 'deviceprotection_No',
       'deviceprotection_Yes', 'techsupport_No', 'techsupport_Yes',
       'streamingtv_No', 'streamingtv_Yes', 'streamingmovies_No',
       'streamingmovies_Yes', 'contract_Month-to-month', 'contract_One year',
       'contract_Two year', 'paperlessbilling_No', 'paperlessbilling_Yes',
       'paymentmethod_Bank transfer (automatic)',
       'paymentmethod_Credit card (automatic)',
       'paymentmethod_Electronic check', 'paymentmethod_Mailed check'],
      dtype='object')

###automl

In [86]:
clf = setup(data = df_train_dumy,
            target = 'churn',
            #numeric_features = ['tenure', 'monthlycharges', 'totalcharges'] 
            #numeric_imputation = 'mean'
            categorical_features = ['seniorcitizen', 'tenure', 'monthlycharges', 'totalcharges',
       'gender_Female', 'gender_Male', 'partner_No', 'partner_Yes',
       'dependents_No', 'dependents_Yes', 'phoneservice_No',
       'phoneservice_Yes', 'multiplelines_No', 'multiplelines_Yes',
       'internetservice_DSL', 'internetservice_Fiber optic',
       'internetservice_No', 'onlinesecurity_No', 'onlinesecurity_Yes',
       'onlinebackup_No', 'onlinebackup_Yes', 'deviceprotection_No',
       'deviceprotection_Yes', 'techsupport_No', 'techsupport_Yes',
       'streamingtv_No', 'streamingtv_Yes', 'streamingmovies_No',
       'streamingmovies_Yes', 'contract_Month-to-month', 'contract_One year',
       'contract_Two year', 'paperlessbilling_No', 'paperlessbilling_Yes',
       'paymentmethod_Bank transfer (automatic)',
       'paymentmethod_Credit card (automatic)',
       'paymentmethod_Electronic check', 'paymentmethod_Mailed check'], # lista das variáveis categóricas
            ignore_features = ['id'], 
            silent = False)

,Description,Value
0,session_id,7752
1,Target,churn
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(5634, 40)"
5,Missing Values,False
6,Numeric Features,0
7,Categorical Features,38
8,Ordinal Features,False
9,High Cardinality Features,False


In [87]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7961,0.8435,0.4981,0.6577,0.5661,0.4364,0.4440,16.717
catboost,CatBoost Classifier,0.7951,0.8443,0.4905,0.6574,0.5609,0.4312,0.4395,15.667
lr,Logistic Regression,0.7936,0.8420,0.5303,0.6386,0.5782,0.4434,0.4475,3.332
rf,Random Forest Classifier,0.7913,0.8297,0.4554,0.6593,0.5374,0.4089,0.4211,5.576
ada,Ada Boost Classifier,0.7913,0.8378,0.5690,0.6214,0.5927,0.4531,0.4547,4.981
lightgbm,Light Gradient Boosting Machine,0.7913,0.8316,0.5256,0.6325,0.5727,0.4366,0.4406,0.488
et,Extra Trees Classifier,0.7892,0.8180,0.4696,0.6465,0.5430,0.4109,0.4202,7.679
xgboost,Extreme Gradient Boosting,0.7837,0.8280,0.5075,0.6159,0.5556,0.4147,0.4185,51.486
ridge,Ridge Classifier,0.7748,0.0000,0.4783,0.6002,0.5307,0.3855,0.3907,1.659
svm,SVM - Linear Kernel,0.7674,0.0000,0.5373,0.5750,0.5437,0.3914,0.3980,1.332


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=7752, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
lr = create_model('lr') 

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8177,0.8501,0.5714,0.6897,0.6250,0.5060,0.5099
1,0.8152,0.8590,0.5048,0.7162,0.5922,0.4773,0.4895
2,0.7797,0.8368,0.4952,0.6047,0.5445,0.4012,0.4046
3,0.7995,0.8371,0.5481,0.6404,0.5907,0.4590,0.4614
4,0.8020,0.8381,0.5673,0.6413,0.6020,0.4709,0.4725
5,0.8020,0.8480,0.5385,0.6512,0.5895,0.4606,0.4642
6,0.8376,0.8723,0.5385,0.7778,0.6364,0.5362,0.5512
7,0.8046,0.8583,0.5769,0.6452,0.6091,0.4794,0.4807
8,0.7868,0.8476,0.5238,0.6180,0.5670,0.4269,0.4294
9,0.7944,0.8542,0.6000,0.6176,0.6087,0.4693,0.4694


In [ ]:
ada = create_model('ada')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8025,0.8462,0.5385,0.6512,0.5895,0.4610,0.4646
1,0.8076,0.8728,0.6250,0.6373,0.6311,0.5009,0.5010
2,0.7975,0.8412,0.5619,0.6344,0.5960,0.4615,0.4630
3,0.8122,0.8720,0.5096,0.6974,0.5889,0.4710,0.4807
4,0.8046,0.8548,0.5865,0.6421,0.6131,0.4827,0.4836
5,0.7919,0.8257,0.6058,0.6058,0.6058,0.4644,0.4644
6,0.8046,0.8506,0.4808,0.6849,0.5650,0.4439,0.4555
7,0.8173,0.8495,0.5481,0.6951,0.6129,0.4955,0.5015
8,0.7868,0.8317,0.5673,0.6020,0.5842,0.4410,0.4413
9,0.8122,0.8696,0.5288,0.6875,0.5978,0.4780,0.4850


In [ ]:
tuned_ada = tune_model(ada)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8203,0.8526,0.5673,0.6941,0.6243,0.5078,0.5122
1,0.8152,0.8761,0.6154,0.6598,0.6368,0.5131,0.5136
2,0.8025,0.8486,0.5619,0.6484,0.6020,0.4716,0.4738
3,0.8122,0.8701,0.5288,0.6875,0.5978,0.4780,0.4850
4,0.8147,0.8601,0.5769,0.6742,0.6218,0.5001,0.5027
5,0.8071,0.8256,0.6346,0.6346,0.6346,0.5036,0.5036
6,0.8147,0.8608,0.4808,0.7246,0.5780,0.4655,0.4816
7,0.8096,0.8465,0.5385,0.6747,0.5989,0.4762,0.4814
8,0.7944,0.8291,0.5865,0.6162,0.6010,0.4626,0.4629
9,0.8147,0.8746,0.5192,0.7013,0.5967,0.4799,0.4890


In [ ]:
tuned_lr = tune_model(lr)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8152,0.8502,0.5619,0.6860,0.6178,0.4975,0.5018
1,0.8152,0.8585,0.5048,0.7162,0.5922,0.4773,0.4895
2,0.7823,0.8363,0.4952,0.6118,0.5474,0.4061,0.4100
3,0.7995,0.8368,0.5385,0.6437,0.5864,0.4554,0.4586
4,0.8020,0.8375,0.5673,0.6413,0.6020,0.4709,0.4725
5,0.8046,0.8490,0.5481,0.6552,0.5969,0.4692,0.4725
6,0.8376,0.8724,0.5385,0.7778,0.6364,0.5362,0.5512
7,0.8096,0.8583,0.5865,0.6559,0.6193,0.4929,0.4943
8,0.7868,0.8475,0.5238,0.6180,0.5670,0.4269,0.4294
9,0.7944,0.8544,0.6000,0.6176,0.6087,0.4693,0.4694


In [88]:
gbc = create_model('gbc')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7722,0.8193,0.4623,0.5976,0.5213,0.3750,0.3803
1,0.7823,0.8187,0.4811,0.6220,0.5426,0.4027,0.4084
2,0.8405,0.8807,0.5755,0.7722,0.6595,0.5582,0.5685
3,0.8020,0.8595,0.5143,0.6667,0.5806,0.4539,0.4604
4,0.8274,0.8604,0.5810,0.7176,0.6421,0.5300,0.5352
5,0.7843,0.8298,0.4286,0.6429,0.5143,0.3827,0.3957
6,0.7716,0.8123,0.4286,0.6000,0.5000,0.3573,0.3658
7,0.8147,0.8452,0.5377,0.7037,0.6096,0.4910,0.4986
8,0.7868,0.8586,0.4434,0.6528,0.5281,0.3968,0.4092
9,0.7792,0.8502,0.5283,0.6022,0.5628,0.4159,0.4175


In [89]:
tuned_gbc = tune_model(gbc)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7747,0.8218,0.5283,0.5895,0.5572,0.4067,0.4078
1,0.7975,0.8154,0.5283,0.6512,0.5833,0.4515,0.4558
2,0.8127,0.8658,0.5660,0.6818,0.6186,0.4958,0.4996
3,0.8071,0.8502,0.5524,0.6667,0.6042,0.4781,0.4818
4,0.8325,0.8490,0.6286,0.7097,0.6667,0.5554,0.5572
5,0.7843,0.8240,0.4667,0.6282,0.5355,0.3990,0.4065
6,0.7690,0.7918,0.4667,0.5833,0.5185,0.3691,0.3730
7,0.7919,0.8223,0.5094,0.6429,0.5684,0.4337,0.4388
8,0.7766,0.8366,0.4717,0.6098,0.5319,0.3884,0.3939
9,0.7741,0.8247,0.5377,0.5876,0.5616,0.4098,0.4106


In [ ]:
'''catb = create_model('catboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8304,0.8637,0.5962,0.7126,0.6492,0.5385,0.5423
1,0.8177,0.8345,0.5192,0.7105,0.6000,0.4856,0.4957
2,0.8076,0.8665,0.6190,0.6436,0.6311,0.5010,0.5012
3,0.7970,0.8225,0.5577,0.6304,0.5918,0.4574,0.4589
4,0.8198,0.8723,0.6346,0.6667,0.6502,0.5290,0.5293
5,0.8071,0.8755,0.5865,0.6489,0.6162,0.4878,0.4889
6,0.7944,0.8155,0.5481,0.6264,0.5846,0.4488,0.4506
7,0.8173,0.8657,0.4904,0.7286,0.5862,0.4746,0.4899
8,0.7995,0.8528,0.5192,0.6506,0.5775,0.4483,0.4532
9,0.8173,0.8562,0.5577,0.6905,0.6170,0.4988,0.5037


In [ ]:
'''tuned_catb = tune_model(catb)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8304,0.8696,0.5962,0.7126,0.6492,0.5385,0.5423
1,0.8203,0.8537,0.5000,0.7324,0.5943,0.4841,0.4986
2,0.8127,0.8690,0.5714,0.6742,0.6186,0.4955,0.4985
3,0.8096,0.8453,0.5192,0.6835,0.5902,0.4692,0.4767
4,0.8274,0.8757,0.6346,0.6875,0.6600,0.5446,0.5454
5,0.8046,0.8797,0.5577,0.6517,0.6010,0.4727,0.4752
6,0.8046,0.8255,0.5481,0.6552,0.5969,0.4692,0.4725
7,0.8198,0.8803,0.5192,0.7200,0.6034,0.4907,0.5017
8,0.8147,0.8636,0.5577,0.6824,0.6138,0.4935,0.4978
9,0.8147,0.8560,0.5385,0.6914,0.6054,0.4868,0.4933


In [90]:
predic = predict_model(tuned_gbc, data = df_test_dumy)
predic

,seniorcitizen,tenure,monthlycharges,totalcharges,gender_Female,gender_Male,partner_No,partner_Yes,dependents_No,dependents_Yes,phoneservice_No,phoneservice_Yes,multiplelines_No,multiplelines_Yes,internetservice_DSL,internetservice_Fiber optic,internetservice_No,onlinesecurity_No,onlinesecurity_Yes,onlinebackup_No,onlinebackup_Yes,deviceprotection_No,deviceprotection_Yes,techsupport_No,techsupport_Yes,streamingtv_No,streamingtv_Yes,streamingmovies_No,streamingmovies_Yes,contract_Month-to-month,contract_One year,contract_Two year,paperlessbilling_No,paperlessbilling_Yes,paymentmethod_Bank transfer (automatic),paymentmethod_Credit card (automatic),paymentmethod_Electronic check,paymentmethod_Mailed check,Label,Score
0,0,23.0,20.00,445.30,0,1,0,1,0,1,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,0,1,0,1,0,0,0,0.9846
1,1,61.0,99.00,5969.30,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,0,0,0.8887
2,0,36.0,84.75,3050.15,0,1,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0.5853
3,0,61.0,61.45,3751.15,1,0,0,1,0,1,1,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,1,0,0,0,0,0.9780
4,0,47.0,20.55,945.70,1,0,1,0,1,0,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0.9861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,0,24.0,49.70,1167.80,0,1,1,0,0,1,0,1,1,0,1,0,0,1,0,1,0,0,1,1,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,0.7601
1405,0,35.0,89.20,3251.30,0,1,1,0,1,0,0,1,1,0,0,1,0,0,1,1,0,1,0,0,1,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0.9366
1406,0,46.0,64.20,3009.50,1,0,0,1,1,0,0,1,1,0,1,0,0,1,0,0,1,1,0,0,1,0,1,1,0,0,0,1,0,1,0,1,0,0,0,0.9727
1407,1,11.0,75.20,775.30,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,0,1,0.5950


In [91]:
predic['Score'].mean()

0.818109723207948

In [92]:
predic['Score'].median()

0.8744

In [ ]:
predic['Score'].std()

0.015189865129068237

In [93]:
pred_val = predic['Label']


In [94]:
from google.colab import files

In [95]:
ids = df_churn_test['id']
output = pd.DataFrame({'id': ids, 'Churn': pred_val})
output.to_csv('submission35gbc.csv', index=False)
files.download('submission35gbc.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>